In [78]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [79]:
installs = pd.read_csv('data/installs.csv.gzip', low_memory=False, compression='gzip', parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()

In [80]:
ventana_1 = installs['2019-04-18':'2019-04-20'].reset_index()

In [81]:
train_ventana_2 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'}).reset_index()

In [82]:
train_ventana_2['created'] = train_ventana_2['created'] - pd.Timestamp('2019-04-21 00:00:00')
train_ventana_2['created'] = train_ventana_2['created'].transform(lambda x: x.total_seconds())

In [83]:
train_ventana_2 = train_ventana_2.merge(ventana_1["ref_hash"].drop_duplicates(), how="right")

In [84]:
ventana_1['created'] = ventana_1['created'] - pd.Timestamp('2019-04-18 00:00:00')
ventana_1['created'] = ventana_1['created'].transform(lambda x: x.total_seconds())


In [85]:
ventana_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160861 entries, 0 to 160860
Data columns (total 18 columns):
created               160861 non-null float64
application_id        160861 non-null int64
ref_type              160861 non-null int64
ref_hash              160861 non-null int64
click_hash            307 non-null object
attributed            160861 non-null bool
implicit              160861 non-null bool
device_countrycode    160861 non-null int64
device_brand          95803 non-null float64
device_model          152978 non-null float64
session_user_agent    156740 non-null object
user_agent            111880 non-null object
event_uuid            35226 non-null object
kind                  35226 non-null object
wifi                  100903 non-null object
trans_id              2867 non-null object
ip_address            160861 non-null int64
device_language       153042 non-null float64
dtypes: bool(2), float64(4), int64(5), object(7)
memory usage: 19.9+ MB


In [86]:
ventana_1.drop(["click_hash", "event_uuid", "kind", "trans_id", "user_agent", "session_user_agent"],axis =1, inplace = True )

In [87]:
ventana_1 = ventana_1.fillna(ventana_1.mode().iloc[0])

In [88]:
ventana_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160861 entries, 0 to 160860
Data columns (total 12 columns):
created               160861 non-null float64
application_id        160861 non-null int64
ref_type              160861 non-null int64
ref_hash              160861 non-null int64
attributed            160861 non-null bool
implicit              160861 non-null bool
device_countrycode    160861 non-null int64
device_brand          160861 non-null float64
device_model          160861 non-null float64
wifi                  160861 non-null bool
ip_address            160861 non-null int64
device_language       160861 non-null float64
dtypes: bool(3), float64(4), int64(5)
memory usage: 11.5 MB


In [89]:
ventana_1 = ventana_1.sort_values(["ref_hash", "created"])

In [90]:
slideing["created"] = ventana_1["created"]
slideing['creac_sig'] = slideing["created"].shift(-1)
slideing

,created,creac_sig
47620,76310.326,76311.966
47622,76311.966,76631.946
47930,76631.946,76636.531
47936,76636.531,192980.258
126935,192980.258,60042.839
33036,60042.839,96156.508
65670,96156.508,92860.475
62665,92860.475,75062.237
46443,75062.237,65400.392
37366,65400.392,73897.181


In [91]:
stop_points = ventana_1.groupby("ref_hash").last().reset_index()["created"]

In [92]:
stop_points[0]

76636.531

In [133]:
j = 0
for i,row in slideing.iterrows():
    if stop_points[j] == row[0]:
        slideing.at[i, "creac_sig"] = 60*60*24*3 - row[0]
        j += 1

In [134]:
ventana_1["censored"] = 0

In [135]:
j = 0
for i,row in ventana_1.iterrows():
    if stop_points[j] == row[0]:
        ventana_1.at[i, "censored"] = 1
        j += 1

**Encodeo las variables categoricas y strings**

In [136]:
"""le_usr_ag = LabelEncoder()
le_ss_usr_ag = LabelEncoder()
ventana_1['user_agent'] = le_usr_ag.fit_transform(ventana_1.user_agent)
ventana_1['session_user_agent'] = le_ss_usr_ag.fit_transform(ventana_1.session_user_agent)"""

"le_usr_ag = LabelEncoder()\nle_ss_usr_ag = LabelEncoder()\nventana_1['user_agent'] = le_usr_ag.fit_transform(ventana_1.user_agent)\nventana_1['session_user_agent'] = le_ss_usr_ag.fit_transform(ventana_1.session_user_agent)"

In [137]:
"""usr_ag_ohe = OneHotEncoder()
ss_usr_ag_ohe = OneHotEncoder()
X = usr_ag_ohe.fit_transform(ventana_1.user_agent.values.reshape(-1,1)).toarray()
Xm = ss_usr_ag_ohe.fit_transform(ventana_1.session_user_agent.values.reshape(-1,1)).toarray()"""

'usr_ag_ohe = OneHotEncoder()\nss_usr_ag_ohe = OneHotEncoder()\nX = usr_ag_ohe.fit_transform(ventana_1.user_agent.values.reshape(-1,1)).toarray()\nXm = ss_usr_ag_ohe.fit_transform(ventana_1.session_user_agent.values.reshape(-1,1)).toarray()'

In [138]:
X_train, X_test, y_train, y_test = \
    train_test_split(ventana_1, slideing["creac_sig"], test_size=0.2, random_state=769)

In [139]:
reg=linear_model.LinearRegression()

In [140]:
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [141]:
pred=reg.predict(X_test)

In [142]:
mean_squared_error(y_test,pred)**0.5


57843.665962169805